# Custom Clients

In [1]:
import os
from pprint import pprint as pp

os.environ["LOG_LEVEL"] = "debug"

LANGGATE_URL = "http://localhost:4000/api/v1"

# If running the langgate server behind Envoy proxy in docker:
# LANGGATE_URL = "http://localhost:10000/api/v1"

# If running and accessing langgate from within a kubernetes cluster:
# LANGGATE_URL = "http://langgate.ns.svc.cluster.local:10000/api/v1"

## Custom Local Registry Client

In [2]:
# If you wan to set a custom config file:
# os.environ["LANGGATE_CONFIG"] = "some_custom_path/langgate_config.yaml"
# os.environ["LANGGATE_CONFIG"] = "some_custom_path/langgate_config.yaml"

### Example 1: Using the SDK's default local registry client

In [3]:
from langgate.sdk import LangGateLocal

# `LangGateLocal.registry` is a singleton
client = LangGateLocal()
models = await client.list_models()  # returns the default LLMInfo schema list
pp(models[0])

2025-06-17 23:18:59 [debug    ] creating_langgate_local_client_singleton
2025-06-17 23:18:59 [debug    ] creating_model_registry_singleton
2025-06-17 23:18:59 [info     ] loaded_model_data              model_count=62 models_data_path=/Users/someuser/langgate/packages/registry/src/langgate/registry/data/default_models.json
2025-06-17 23:18:59 [info     ] loaded_config                  config_path=/Users/someuser/langgate/examples/langgate_config.yaml
2025-06-17 23:18:59 [debug    ] initialized_model_registry_singleton config_path=/Users/someuser/langgate/examples/langgate_config.yaml env_file_exists=False env_file_path=/Users/someuser/langgate/examples/.env models_data_path=/Users/someuser/langgate/packages/registry/src/langgate/registry/data/default_models.json
2025-06-17 23:18:59 [debug    ] initialized_base_local_registry_client
2025-06-17 23:18:59 [debug    ] initialized_local_registry_client_singleton
2025-06-17 23:18:59 [debug    ] resolved_config_path           exists=True path=/

### Example 2: Using the standalone default local registry client
The registry can be installed without the SDK if you do not need to use any other SDK features such as transforming paramaters.
It can be installed separately with:
```bash
uv add langgate[registry]
```
or with pip:
```bash
pip install langgate[registry]
```

In [4]:
from langgate.registry.local import LocalRegistryClient

# The concrete `LocalRegistryClient` class is a singleton
client = LocalRegistryClient()
models = await client.list_models()  # returns the default LLMInfo schema list
pp(models[0])

LLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_response_schema=True, supports_system_messages=True, supports_tool_choice=True), context_window=ContextWindow(max_input_tokens=1047576, max_output_tokens=32768), provider=ModelProvider(id='openai', name='OpenAI', description=None), updated_dt=datetime.datetime(2025, 6, 17, 22, 18, 59, 740302, tzinfo=datetime.timezone.utc))


### Example 3: Subclassing BaseLocalRegistryClient with custom schema

In [5]:
from langgate.registry.local import BaseLocalRegistryClient
from langgate.core.models import LLMInfo


class CustomLLMInfo(LLMInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomLocalRegistryClient(BaseLocalRegistryClient[CustomLLMInfo]):
    """Custom Local registry client with CustomLLMInfo schema."""

    # This is not a singleton unless you implement it as such.


custom_client = CustomLocalRegistryClient()
custom_models = (
    await custom_client.list_models()
)  # Typed and validated as list[CustomLLMInfo]
pp(custom_models[0])

2025-06-17 23:19:37 [debug    ] reusing_registry_singleton     initialized=True
2025-06-17 23:19:37 [debug    ] initialized_base_local_registry_client
2025-06-17 23:19:37 [debug    ] refreshing_model_cache        
2025-06-17 23:19:37 [debug    ] refreshed_model_cache          model_count=5
CustomLLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_response_schema=True, supports_system_messages=True, supports_tool_choice=True), context_windo

### Example 4: Using the base class directly with type parameter

In [6]:
# This is not a singleton.
local_client_with_custom_schema = BaseLocalRegistryClient[CustomLLMInfo](
    model_info_cls=CustomLLMInfo
)
direct_models = (
    await local_client_with_custom_schema.list_models()
)  # Typed and validated as list[CustomLLMInfo]

pp(direct_models[0])

2025-06-17 23:19:39 [debug    ] reusing_registry_singleton     initialized=True
2025-06-17 23:19:39 [debug    ] initialized_base_local_registry_client
2025-06-17 23:19:39 [debug    ] refreshing_model_cache        
2025-06-17 23:19:39 [debug    ] refreshed_model_cache          model_count=5
CustomLLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_response_schema=True, supports_system_messages=True, supports_tool_choice=True), context_windo

## Custom HTTP Registry Client

In [7]:
from pydantic import SecretStr

# Optional: Set API key (if your LangGate service requires authentication)
api_key = SecretStr("your_api_key")

### Example 1: Using the default http client

In [8]:
from langgate.client.http import HTTPRegistryClient

# The concrete `HTTPRegistryClient` class is a singleton
client = HTTPRegistryClient(base_url=LANGGATE_URL, api_key=api_key)
models = await client.list_models()  # returns the default LLMInfo schema list
pp(models[0])

2025-06-17 23:19:44 [debug    ] creating_http_registry_client_singleton
2025-06-17 23:19:44 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 model_info_cls=<class 'langgate.core.models.LLMInfo'>
2025-06-17 23:19:44 [debug    ] initialized_http_registry_client_singleton
2025-06-17 23:19:44 [debug    ] refreshing_model_cache        
2025-06-17 23:19:44 [debug    ] refreshed_model_cache          model_count=58
LLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, sup

### Example 2: Subclassing BaseHTTPRegistryClient with custom schema

In [9]:
from langgate.client.http import BaseHTTPRegistryClient
from langgate.core.models import LLMInfo


class CustomLLMInfo(LLMInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomHTTPClient(BaseHTTPRegistryClient[CustomLLMInfo]):
    """Custom HTTP client with CustomLLMInfo schema."""

    # This is not a singleton unless you implement it as such.


custom_client = CustomHTTPClient(LANGGATE_URL, api_key=api_key)
custom_models = (
    await custom_client.list_models()
)  # Typed and validated as list[CustomLLMInfo]
pp(custom_models[0])

2025-06-17 23:19:46 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 model_info_cls=<class '__main__.CustomLLMInfo'>
2025-06-17 23:19:46 [debug    ] refreshing_model_cache        
2025-06-17 23:19:46 [debug    ] refreshed_model_cache          model_count=58
CustomLLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_response_schema=True, supports_system_messages=True, sup

### Example 3: Using the base class directly with type parameter

In [10]:
# This is not a singleton.
client_with_custom_schema = BaseHTTPRegistryClient[CustomLLMInfo](
    base_url=LANGGATE_URL, api_key=api_key, model_info_cls=CustomLLMInfo
)
direct_models = (
    await client_with_custom_schema.list_models()
)  # Typed and validated as list[CustomLLMInfo]

pp(direct_models[0])

2025-06-17 23:19:48 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 model_info_cls=<class '__main__.CustomLLMInfo'>
2025-06-17 23:19:48 [debug    ] refreshing_model_cache        
2025-06-17 23:19:48 [debug    ] refreshed_model_cache          model_count=58
CustomLLMInfo(id='openai/gpt-4.1', name='GPT-4.1', provider_id='openai', description="GPT-4.1 is the latest iteration of OpenAI's flagship model with improved capabilities across all domains.", costs=ModelCost(input_cost_per_token=Decimal('0.000002'), output_cost_per_token=Decimal('0.000008'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('5E-7')), capabilities=ModelCapabilities(supports_tools=True, supports_parallel_tool_calls=True, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_response_schema=True, supports_system_messages=True, sup